# An Introduction to Optimisation via Simulation.  How to choose the best setup for a system.
## A tutorial for the Simulation Workshop 2020

**Dr Christine S.M Currie and Tom Monks**

This tutorial introduces Ranking and Selection procedures for stochastic computer simulation.  We focus on in difference zone and optimal computer budget allocation procedures. The procedures are:

* KN and KN++
* OCBA and OCBA-m

In [1]:
import numpy as np

In [2]:
from ovs.toy_models import (custom_guassian_model,
                            guassian_sequence_model,
                            random_guassian_model,
                            ManualOptimiser)

## Simulation Models

To keep the focus of this tutorial on Ranking and Selection procedures and what they do, and how they perform in practice, the models we introduce are deliberately simple.   The following sections describe how to create a a set of simulation models where the outputs are independent guassian distributions.  There are three options:

* the means of these outputs distributions follow a sequence (e.g. 1, 2, 3, 4, 5) variance is 1.0. 
* the means and variances are user specified.
* the means and variances are randomly generated.

### Creating a simple sequence of normal distributions

A simple model can be created using the `guassian_sequance_model()` function.  This creates a simulation model where the output of each design follows a $N - (\mu_{i}, 1.0)$ :

The function accepts three keyword arguments:

* **start** - int, the first mean in the sequence (inclusive)
* **end** - int, the last mean int the sequence (inclusive)
* **step** - int, the difference between mean i and mean i + 1.

For example, the following code creates a simulation model with 10 designs with means 1 to 10 and unit variance.

```python
model = guassian_sequence_model(1, 10)
```

To create a simulation model with 5 designs where $\mu_{i+1} - \mu_i = 2 $ :

```python
model = guassian_sequence_model(1, 10, step=2)
```

### Creating a custom model with known designs
Instead of a sequence of normal distributions with unit variance, it is possible to create a custom set of designs with varying variances. Use the `custom_guassian_model` function for this task.  For example to create a custom set of designs: 

```python
means = [5, 8, 1, 2, 1, 7]
variances = [0.1, 1.2, 1.4, 0.3, 0.8]

custom_model = custom_guassian_model(means, variances)
```

The following code demonstrates how to create a sequence of 100 designs with variances that are 10% of the mean. 

```python
n_designs = 100
means = [i for i in range(n_designs+1)]
variances = [j*0.1 for j in range(n_designs+1)]

custom_model = custom_guassian_model(means, variances)
```


### Creating a model with randomly sampled designs

To create a model with a set of unknown designs (within a specified mean and variance tolerance) use 

```python
mean_low = 1.0
mean_high = 15.0
var_low = 0.1
var_high = 2.0
n_designs = 15

model = random_guassian_model(mean_low, mean_high, var_low, var_high, n_designs)
```

Where:
* **mean_low** - float, a lower bound on the means of the output distributions
* **mean_high** - float, an upper bound on the means
* **var_low** - float, a lower bound on the variance of the output distributions
* **var_high** - float, an upper bound on the variances.
* **n_designs** - int, the number of designs to create.

## A manual optimisation framework

Before using the **Optimisation via Simulation** (OvS) procedures, it is recommended that you get a feel for the framework in which the OvS procedures operate.  To do this we will create some models and explore them using a `ManualOptimiser`.  This allows the user to run independent and multiple replications of the model yourself independent of any algorithm.  The `ManualOptimiser` keeps track of the means, variances and number of replications run for each design.

A `ManualOptimiser` object requires two parameters when it is created.  

* model - object, e.g. a model that is a sequence of normal distributions
* n_designs - the number of designs to be considered.

In [3]:
manual_opt = ManualOptimiser(model=guassian_sequence_model(1, 10),
                             n_designs=10)

* We can print the optimiser object to help us remember what parameters we set.

In [4]:
print(manual_opt)

ManualOptimiser(model=BanditCasino(), n_designs=10, verbose=False)


* Follow Law and Kelton's advice and run 3 initial replications of each design.

In [5]:
manual_opt.simulate_designs(replications=3)

The optimiser keeps track of the allocation of replications across each design.  For efficiency it doesn't store each individual observation, but it does compute a running mean and variance.

* Let's have a look at the replication allocation between and results of each design.

In [6]:
manual_opt.allocations

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [7]:
np.set_printoptions(precision=1) # this is a hack to view at 1 decimal place.
manual_opt.means

array([0.9, 1.6, 3.5, 3.5, 4.1, 6.6, 6.3, 8.7, 9.1, 9.4])

In [8]:
#TM to do: this should be variances not sigmas
manual_opt.sigmas

array([3.2, 1.5, 1.3, 0.6, 0.2, 1. , 0.1, 0.6, 0.8, 1. ])

Now let's run 1 additional replication of the top 3 designs. 
Note - in python arrays are **zero indexed**.  This means that design 1 has index 0.

In [9]:
manual_opt.simulate_designs(design_indexes=[7, 8, 9], replications=1)

In [10]:
manual_opt.allocations

array([3, 3, 3, 3, 3, 3, 3, 4, 4, 4], dtype=int32)

In [11]:
manual_opt.simulate_designs(design_indexes=[7, 8, 9], replications=2)

In [12]:
manual_opt.means

array([0.9, 1.6, 3.5, 3.5, 4.1, 6.6, 6.3, 8.2, 8.8, 9.3])

In [13]:
manual_opt.sigmas

array([3.2, 1.5, 1.3, 0.6, 0.2, 1. , 0.1, 0.6, 0.5, 0.4])

### Manual Optimisation of a model with unknown means.

Now have a go yourself.  This time create a model with random designs.  Run as many replications of each design as you think is neccessary to make a decision abouit which design is best.  Here we define best as the design with the largest mean value.

In [14]:
rnd_model = random_guassian_model(mean_low=5, mean_high=25, 
                                  var_low=0.5, var_high=2.5,
                                  n_designs=10)

manual_opt = ManualOptimiser(model=rnd_model, n_designs=10)

In [15]:
#insert your code here...


# Indifference Zone Ranking and Selection

## Procedure **KN**
The first Ranking and Selection (R&S) algorithm we will explore is **Procedure KN** developed by Kim and Nelson. This is an *Indifference Zone* (IZ) approach to R&S.  IZ procedures exploit the the idea that the performance of one or more of the sub-optimal designs are in fact so close to the best performing system that a user does not care which one is chosen (the decision maker is said to be *indifferent* to this choice).  To do this a user must specify a quantity $\delta$.  Procedure KN provides a theorectical guarantee to select the best system or a system within $\delta$ of the best with probability of $1 - \alpha$.

A key feature of KN is that it only estimates the variance of each design once.  This happens after an initial number of replications (specified by the user) $n_0$.  

To run Kim and Nelson's R&S procedure KN, create an instance of `ovs.indifference_zone.KN`

An object of type KN takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **delta** - float, the indifference zone
* **alpha** - float, $PCS = 1-\alpha$ (default=0.05)
* **n_0** - int, $n_0$ the number of initial replications (default=2)

In [17]:
from ovs.indifference_zone import KN, KNPlusPlus

### Example 1.  A simple sequence of normal distributions.

The first problem test KN against is selecting the largest mean from a sequence of 10 normal distributions (means raning from 1 to 10) with unit variance.  We want a $PCS = 0.95$ and are indifference to designs that are within an average of 1.0 of the best.  

We will follow Law's longstanding advise of setting $n_0 = 5$ i.e 5 initial replications of each design.

#### Setting up KN

In [108]:
n_designs = 10

#first we create the simulation model. 
model = guassian_sequence_model(1, n_designs)

#then we create the KN R&S object and pass in our parameters.
#note that this doesn't run the optimisation yet.
kn = KN(model=model, 
        n_designs=n_designs, 
        delta=1.0, 
        alpha=0.05, 
        n_0=5)

#### Running KN
Now that we have created the KN object we can run the solver at any time we choose.  To do this call the `KN.solve()` method.  This method returns a design within the indifference zone $(1 - alpha$) of the time.

In [109]:
#this runs KN
best_design = kn.solve()

#print out the results
print(f'best design\t{best_design}')
print(f'allocations\t{kn._allocations}')
print(f'total reps\t{kn._allocations.sum()}')

best design	[9]
allocations	[ 5  5  5  5  5  5  5 10 18 19]
total reps	82


In general you should see that KN simulates the top 2-3 designs the most with the lower performing designs eliminated early on (possibly after the initial stage of replication).

## Procedure **KN++**

Kim and Nelson's KN++ procedure is an enhancement on KN.  KN++ introduces an **update** step that recalculates the variance of the differences between designs.  

To run procedure KN++, create an instance of `ovs.indifference_zone.KNPlusPlus`

### Example 2. Solving the simple problem using KN++

We will run KN++ on the same simulation model as KN.  In general you should see that KN++ uses less replication to find a design within the indifference zone.

In [110]:
knpp = KNPlusPlus(model=model, 
                  n_designs=n_designs, 
                  delta=1.0, 
                  alpha=0.05, 
                  n_0=5)

best_design = knpp.solve()
print(f'best design\t{best_design}')
print(f'allocations\t{knpp._allocations}')
print(f'total reps\t{knpp._allocations.sum()}')

best design	[9]
allocations	[ 5  5  5  5  5  5  6  8 10 11]
total reps	65


## Exercise 1. Comparing KN and KN++ on a larger design.

Your task is to compare KN and KN++ optimising a simulation model with 100 competing designs.  These designs are a sequence of 100 normal distributions with unit variance.  The objective is to find the design with the maximum mean.

What do you notice about the replications required from each method?

In [ ]:
#the code to set up the simulation model has been provided below.
N_DESIGNS = 100
DELTA = 1.0
ALPHA = 0.05
N_0 = 5

#first we create the simulation model. 
model = guassian_sequence_model(1, N_DESIGNS)

In [ ]:
#your code goes here...
# hint 1. you need to create KN and KNPlusPlus object, pass in the simulation model and run them.  
# hint 2. try running the models a few times.  



## Exercise 2: A harder optimisation problem

We will now explore how the IZ procedures perform on a slightly harder optimisation problem where the mean of the designs are closer together and have differing variances.

The problem is: 

```python
means = [4, 4.1, 4.2, 4, 4.1, 4.3, 4, 4.1, 4.2, 4.2]
variances = [1, 1, 1, 0.1, 0.1, 10, 10, 10, 10, 0.1]
```

The 'optimal' mean design is at index 5 (4.3).  

Your task is to compare the performance of KN and KN++ given the following parameters:

```python
N_DESIGNS = 10
N_0 = 10
DELTA = 0.15
ALPHA = 0.05
```

Questions: 
* Which designs require the most replicatin effort?
* Are there any differences in designs selected and the number of replications needed?
* What happens if you try difference values of $n_0$ e.g. 20 and 50?
* What happens if you change $\alpha$ to 0.1? 

**RUNTIME WARNING!** This problem requires many more replications.  It will take a few seconds for the procedures to return a design.

In [141]:
N_DESIGNS = 10
N_0 = 20
DELTA = 0.15
ALPHA = 0.05

means = [4, 4.1, 4.2, 4, 4.1, 4.3, 4, 4.1, 4.2, 4.2]
variances = [1, 1, 1, 0.1, 0.1, 10, 10, 10, 10, 0.1]

guass_model = custom_guassian_model(means, variances)

#print out which design indexes are in the indifference zone
print(np.where(4.3 - np.array(means) <= DELTA)[0])

[2 5 8 9]


In [142]:
kn = KN(model=guass_model, 
        n_designs=N_DESIGNS, 
        delta=DELTA, 
        alpha=ALPHA, 
        n_0=N_0)

best_design = kn.solve()

#print out the results
print(f'best design\t{best_design}')
print(f'allocations\t{kn._allocations}')
print(f'total reps\t{kn._allocations.sum()}')

best design	[8]
allocations	[   83   759 10387    20    20    81 26453 32033 32034    20]
total reps	101890


In [143]:
knpp = KNPlusPlus(model=guass_model, 
                  n_designs=N_DESIGNS, 
                  delta=DELTA, 
                  alpha=ALPHA, 
                  n_0=N_0)

best_design = knpp.solve()

#print out the results
print(f'best design\t{best_design}')
print(f'allocations\t{knpp._allocations}')
print(f'total reps\t{knpp._allocations.sum()}')

best design	[5]
allocations	[  122    77   270    20    20 44553  6909 27347 44552 15933]
total reps	139803


## Optimal Computing Budget Allocation (OCBA)

An object of type OCBA takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [ ]:
from ovs.fixed_budget import OCBA, OCBAM

In [ ]:
n_designs = 10
guass_model = guassian_sequence_model(1, n_designs)

rnd_model = random_guassian_model(mean_low=5, mean_high=25, 
                                  var_low=0.5, var_high=2.5,
                                  n_designs=10)

ocba = OCBA(model=rnd_model, 
            n_designs=n_designs, 
            budget=500, 
            delta=5, 
            n_0=5, 
            obj='max')

In [ ]:
print(ocba)

call the `solve()` method to run the optimisation

In [ ]:
results = ocba.solve()
print('best design:\t{}'.format(results))
print('allocations:\t{}'.format(ocba._allocations))
print('total reps:\t{}'.format(ocba._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocba._means))
print('vars:\t\t{0}'.format(ocba._vars))

#### A harder problem...

In [ ]:
#try varying the budget, delta and n_0.  What is the effect?
#e.g try a budge

means = [4, 4.1, 4.2, 4, 4.1, 4.2, 4, 4.1, 4.2, 4.3]
variances = [1, 1, 1, 0.1, 0.1, 0.1, 10, 10, 10, 10]

guass_model = custom_guassian_model(means, variances)

ocba = OCBA(model=guass_model, 
            n_designs=n_designs, 
            budget=10000, 
            delta=5, 
            n_0=20, 
            obj='max')

In [ ]:
ocba.reset()
results = ocba.solve()
print('best design:\t{}'.format(results))
print('allocations:\t{}'.format(ocba._allocations))
print('total reps:\t{}'.format(ocba._allocations.sum()))


np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocba._means))

## Optimal Computing Budget Allocation Top M (OCBA-m)

OCBA-m extended OCBA to identify the top m designs.  

An object of type `OCBAM` takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **m** - int, $m$ the number of top designs to return.  m must be >=2.  For m = 1 see OCBA.
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [ ]:
n_designs = 10
guass_model = guassian_sequence_model(1, n_designs+1)

ocbam = OCBAM(model=guass_model, 
              n_designs=n_designs, 
              budget=500, 
              delta=5, 
              n_0=5, 
              m=2,
              obj='max')

In [ ]:
print(ocbam)

In [ ]:
results = ocbam.solve()
print('best designs:\t{}'.format(np.sort(results)))
print('allocations:\t{}'.format(ocbam._allocations))
print('total reps:\t{}'.format(ocbam._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocbam._means))
print('vars:\t\t{0}'.format(ocbam._vars))
print('SEs:\t\t{0}'.format(ocbam._ses))

# Evaluation of the methods

In [ ]:
from ovs.evaluation import Experiment

In [ ]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.1, 
        alpha=0.2, 
        n_0=5)

exp = Experiment(env=environment, procedure=kn, best_index=9, replications=10)

In [ ]:
results = exp.execute()

In [ ]:
results.p_correct_selections

In [ ]:
results.selections

In [ ]:
results.correct_selections

In [ ]:
designs = guassian_bandit_sequence(1, 101)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=200, 
            delta=10, 
            n_0=5, 
            min=False)

exp = Experiment(env=environment, procedure=ocba, best_index=9, replications=50)

In [ ]:
results = exp.execute()

In [ ]:
results.p_correct_selections

In [ ]:
results.selections

In [ ]:
results.correct_selections

## Try different budgets

In [ ]:
from ovs.evaluation import GridExperiment

In [ ]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

param_grid = {'model':[environment],
              'budget':[100, 200, 300, 400, 500], 
              'n_designs':[len(designs)],
              'delta':[1, 5,10],
              'n_0':[1, 5],
              'min':[False]
              }


exp = GridExperiment(agent=ocba, 
                     environment=environment, 
                     param_grid=param_grid,
                     best_index=9,
                     replications=1000)

In [ ]:
results = exp.fit()

In [ ]:
results